# Red Neuronal

## Librerias

In [23]:
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import re
import unicodedata
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sqlalchemy import create_engine
import json
import psycopg2
from psycopg2.extras import execute_batch

## Cargar Modelo y Utilidades

In [24]:
# -------------------
# 1. Cargar modelo y utilidades
# -------------------
model = tf.keras.models.load_model("modelo_categorias (2).keras")

with open(r"C:\Users\MoisesEugenioNavaMar\red_neuronal\Aplicativo-Categorizacion\tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open(r"C:\Users\MoisesEugenioNavaMar\red_neuronal\Aplicativo-Categorizacion\labelencoder.pkl", "rb") as f:
    le = pickle.load(f)

## Normaliza Texto

In [25]:
# -------------------
# 2. Función de normalización (misma que en el entrenamiento)
# -------------------
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

## CArgar Datos

In [26]:
def conection_bd():
    try:
        host = "directob2b.postgres.database.azure.com"
        port = "5432"
        database = "ecommerce"
        user = "directob2b"
        password = "Noctua2025$"
        engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
        print("Conexión exitosa 🚀")
        return engine
    except Exception as e:
        print("Error al conectar con SQLAlchemy:", e)
        return None

In [27]:
engine = conection_bd()
try:
    query = """SELECT tp.csku,
       tp.cnombre,
       tp.cdescripcion,
       tp.cmarca,
       tnc.id_subcategoria as id_subcategoria,
       tnc.nombre_subcategoria as subcategoria
FROM tbl_producto as tp
  LEFT JOIN tbl_nueva_subcategoria as tnc
    ON tnc.id_subcategoria = tp.id_nueva_subcategoria"""
    df_productos = pd.read_sql(query, engine)
    print('Catálogo obtenido')
except Exception as e:
    print("Error al ejecutar la consulta:", e)


engine.dispose()

Conexión exitosa 🚀
Catálogo obtenido


In [28]:
df_productos.sample()

,csku,cnombre,cdescripcion,cmarca,id_subcategoria,subcategoria
24040,MMP-LICENSE-B-PRO,"Licencia MMP PRO de dispositivos Mimosa, nube ...","Licencia MMP PRO de dispositivos Mimosa, nube ...",Mimosa Networks,13.0,Software y Garantias


## Filtrar a No Categorizados

In [29]:
df_productos = df_productos[df_productos['subcategoria'].isna()]

In [30]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16848 entries, 9 to 76681
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   csku             16848 non-null  object 
 1   cnombre          16848 non-null  object 
 2   cdescripcion     16848 non-null  object 
 3   cmarca           16847 non-null  object 
 4   id_subcategoria  0 non-null      float64
 5   subcategoria     0 non-null      object 
dtypes: float64(1), object(5)
memory usage: 921.4+ KB


## Normalizar Texto 

In [31]:
# -------------------
# 4. Unir columnas de texto para el modelo
# -------------------
df_productos["texto"] = (
    df_productos["cnombre"].fillna("").astype(str) + " " +
    df_productos["cdescripcion"].fillna("").astype(str) + " " +
    df_productos["cmarca"].fillna("").astype(str)
)

In [32]:
# Normalizar
df_productos["texto_norm"] = df_productos["texto"].apply(normalize_text)

## Tokenizar y Predecir

In [33]:
# -------------------
# 5. Tokenizar y predecir
# -------------------
secuencias = tokenizer.texts_to_sequences(df_productos["texto_norm"])
X = pad_sequences(secuencias, maxlen=200)

y_pred = model.predict(X, verbose=0)
categorias_pred = le.inverse_transform(np.argmax(y_pred, axis=1))

## Guardar Resultado

In [34]:
# -------------------
# 6. Guardar resultado en el DataFrame
# -------------------
df_productos["categoria_predicha"] = categorias_pred


In [35]:
df_productos.sample(5)

,csku,cnombre,cdescripcion,cmarca,id_subcategoria,subcategoria,texto,texto_norm,categoria_predicha
558,PST-H080-01,Nivel Magnético de Aluminio Tipo Torpedo de 9”...,Nivel Magnético de Aluminio Tipo Torpedo de 9”...,Precision,NaN,None,Nivel Magnético de Aluminio Tipo Torpedo de 9”...,nivel magnetico de aluminio tipo torpedo de 9 ...,Energía y componentes eléctricos
73733,DS-C30S-02HI/4K(V2),Tarjeta de 2 Entradas HDMI 4K / Compatible con...,Tarjeta de 2 Entradas HDMI 4K / Compatible con...,Hikvision,NaN,None,Tarjeta de 2 Entradas HDMI 4K / Compatible con...,tarjeta de 2 entradas hdmi 4k compatible con c...,Electrodomésticos
66656,WGW1002G,"Gateway GSM 3G, con 2 puertos para SIM, compat...","Gateway GSM 3G, con 2 puertos para SIM, compat...",Openvox,NaN,None,"Gateway GSM 3G, con 2 puertos para SIM, compat...",gateway gsm 3g con 2 puertos para sim compatib...,Equipamiento de Redes y Conectividad
26029,EP-CAMLOCK-400A-M-WHITE,Conector camlock | 400 AMPERES | MACHO | color...,Conector camlock | 400 AMPERES | MACHO | color...,Epcom,NaN,None,Conector camlock | 400 AMPERES | MACHO | color...,conector camlock 400 amperes macho color blanc...,Canalización
55621,88001-0084,CUBIERTAS PARA MOTOR BK CAME,CUBIERTAS PARA MOTOR BK CAME,Came,NaN,None,CUBIERTAS PARA MOTOR BK CAME CUBIERTAS PARA M...,cubiertas para motor bk came cubiertas para mo...,Energía y componentes eléctricos


In [36]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16848 entries, 9 to 76681
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   csku                16848 non-null  object 
 1   cnombre             16848 non-null  object 
 2   cdescripcion        16848 non-null  object 
 3   cmarca              16847 non-null  object 
 4   id_subcategoria     0 non-null      float64
 5   subcategoria        0 non-null      object 
 6   texto               16848 non-null  object 
 7   texto_norm          16848 non-null  object 
 8   categoria_predicha  16848 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


## Actualizar Info

In [37]:
# Conexión a la base de datos
engine = conection_bd()

Conexión exitosa 🚀


In [38]:
# Prepara los datos: (categoria_predicha, csku)
updates = list(zip(df_productos['categoria_predicha'], df_productos['csku']))

In [39]:
# Limpieza opcional de caracteres nulos
updates = [(cat.replace('\u0000', '') if isinstance(cat, str) else cat, sku) for cat, sku in updates]

In [40]:
# Conexión cruda para psycopg2
conn = engine.raw_connection()
cursor = conn.cursor()

In [41]:
# Consulta SQL para actualizar
query = """
UPDATE tbl_producto
SET categoria_predicha = %s
WHERE csku = %s;
"""

In [42]:
# Ejecuta en lote para mejorar rendimiento
execute_batch(cursor, query, updates)

In [43]:
# Confirmar cambios
conn.commit()

# Cerrar conexiones
cursor.close()
conn.close()
engine.dispose()